In [1]:
import pandas as pd
from datetime import datetime
from dateutil import relativedelta

API_SIDRA_IPCA_MENSAL_ULTIMOS_N = 'https://apisidra.ibge.gov.br/values/t/7060/n1/all/v/63,66/p/last%20{}/c315/all/d/v63%202,v66%204'

# Coleta dos dados

Para nao ficar carregando toda vez os dados historicos, deixei salvo os dados até 09/2022. Quando alguem mais para frente for usar, ai sim o python carrega da API.

## Funcoes para fazer o donwload, se tiver novas divulgacoes, consolidacao e limpeza dos dados

In [2]:
def download_ultimos_ipcas(meses_pra_coletar):
    api_sidra_ultimos_ipcas = API_SIDRA_IPCA_MENSAL_ULTIMOS_N.format(meses_pra_coletar)

    df_ultimos_ipcas = pd.read_json(api_sidra_ultimos_ipcas)

    return df_ultimos_ipcas

In [3]:
def carregar_ultimos_ipcas():
    dt_hoje = datetime.today().replace(day=1)
    dt_ultima_na_base = datetime.strptime('01/09/2022', '%d/%m/%Y')

    meses_pra_coletar = relativedelta.relativedelta(dt_hoje, dt_ultima_na_base)
    meses_pra_coletar = meses_pra_coletar.months - 1

    if meses_pra_coletar:
        df_ipca_mensal_recente = download_ultimos_ipcas(meses_pra_coletar)
        return df_ipca_mensal_recente
    else:
        return pd.DataFrame()

In [4]:
def limpar_ipca_mensal(df_ipca_bruto):
    df_ipca_completo = df_ipca_bruto.copy()

    # Ajuste das colunas
    df_ipca_completo.columns = df_ipca_completo.iloc[0]
    df_ipca_completo.drop(0, inplace=True)

    # Arrumar os nomes das colunas
    equivalencia_colunas = {
        'Nível Territorial (Código)': 'cod_nivel_territorio',
        'Nível Territorial': 'nivel_territorio',
        'Unidade de Medida (Código)': 'cod_unidade_medida',
        'Unidade de Medida': 'unidade_medida',
        'Valor': 'valor',
        'Brasil (Código)': 'cod_pais',
        'Brasil': 'pais',
        'Variável (Código)': 'cod_variavel',
        'Variável': 'variavel',
        'Mês (Código)': 'cod_mes',
        'Mês': 'mes',
        'Geral, grupo, subgrupo, item e subitem (Código)': 'cod_item',
        'Geral, grupo, subgrupo, item e subitem': 'item'
    }

    df_ipca_completo.columns = [equivalencia_colunas[coluna] for coluna in df_ipca_completo.columns]

    # Pegar apenas as colunas uteis
    df_ipca_completo = df_ipca_completo[[
        'variavel',
        'cod_mes',
        'cod_item',
        'item',
        'valor'
    ]]

    # Ajustar valores das colunas:
    # Variavel: variacao e peso mensais
    df_ipca_completo.replace({
        'variavel': {'IPCA - Variação mensal': 'variacao', 'IPCA - Peso mensal': 'peso'}
    }, inplace=True)

    # Carregando a equivalencia dos nomes e codigos dos itens
    df_equivalencia_codigos_itens = pd.read_pickle('equivalencia_codigos_itens.pickle')
    df_equivalencia_codigos_itens['cod_item_ibge'] = df_equivalencia_codigos_itens['cod_item_ibge'].astype(str)

    df_ipca_completo = df_ipca_completo.merge(
        df_equivalencia_codigos_itens[['cod_item_ibge', 'cod_item', 'nome_item']],
        'left',
        left_on='cod_item',
        right_on='cod_item_ibge',
        suffixes=['_apagar', '']
    )

    # Arrumando a coluna de data/mes
    df_ipca_completo['data'] = pd.to_datetime(df_ipca_completo['cod_mes'], format='%Y%m')

    # Tirar as colunas inuteis e re ordenar
    df_ipca_completo = df_ipca_completo[[
        'variavel',
        'data',
        'item',
        'cod_item',
        'nome_item',
        'valor'
    ]]

    return df_ipca_completo

## Funcao unica que sequencia todo o processo e retorna o DataFrame prontinho

In [5]:
def carregar_ipca_mensal_completo():
    df_ipca_ate_2022_10 = pd.read_pickle('ipca_mensal_ate_2022-09.pickle')

    df_ultimos_ipcas = carregar_ultimos_ipcas()

    if not df_ultimos_ipcas.empty:
        df_ipca_mensal_completo = pd.concat([df_ipca_ate_2022_10, df_ultimos_ipcas])
    else:
        df_ipca_mensal_completo = df_ipca_ate_2022_10.copy()

    df_ipca_mensal_completo = limpar_ipca_mensal(df_ipca_mensal_completo)

    return df_ipca_mensal_completo

In [6]:
carregar_ipca_mensal_completo()

,variavel,data,item,cod_item,nome_item,valor
0,variacao,2020-01-01,Índice geral,0,Índice geral,0.21
1,variacao,2020-01-01,1.Alimentação e bebidas,1000000,Alimentação e bebidas,0.39
2,variacao,2020-01-01,11.Alimentação no domicílio,1100000,Alimentação no domicílio,0.20
3,variacao,2020-01-01,"1101.Cereais, leguminosas e oleaginosas",1101000,"Cereais, leguminosas e oleaginosas",0.75
4,variacao,2020-01-01,1101002.Arroz,1101002,Arroz,1.31
...,...,...,...,...,...,...
30157,peso,2022-09-01,9101010.Tv por assinatura,9101010,Tv por assinatura,0.3535
30158,peso,2022-09-01,9101018.Acesso à internet,9101018,Acesso à internet,0.5001
30159,peso,2022-09-01,9101019.Aparelho telefônico,9101019,Aparelho telefônico,0.9463
30160,peso,2022-09-01,9101115.Serviços de streaming,9101115,Serviços de streaming,0.0744


# Calculo da inflacao individual